**Attention:** Note that the audio matrix is opposed to it in the lecture notes! That means the size[1] of audio matrix is the number of speakers in "cocktail" party.

In [1]:
### Independent Components Analysis
###
### This program requires a working installation of:
###
### On Mac:
###     1. portaudio: On Mac: brew install portaudio
###     2. sounddevice: pip install sounddevice
###
### On windows:
###      pip install pyaudio sounddevice
###

import sounddevice as sd
import numpy as np

In [2]:
Fs = 11025

def normalize(dat):
    return 0.99 * dat / np.max(np.abs(dat))

def load_data():
    mix = np.loadtxt('mix.dat')
    return mix # each column corresponding to one of the mixed signals xi

def play(vec):
    sd.play(vec, Fs, blocking=True)

In [3]:
# first define its cumulative distribution function/cdf
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [4]:
def unmixer(X):
    M, N = X.shape#MxN
    W = np.eye(N) #NxN, memory error if eye(M)??

    anneal = [0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01, 0.01,
              0.005, 0.005, 0.002, 0.002, 0.001, 0.001]
    print('Separating tracks ...')
    ######## Your code here ##########
    #update the parameter
    for alpha in anneal:
        print("we are at alpha: {0}".format(alpha))
        for x in X:
            # compute array and array to get matrix
            # method 1: use np.mat() changing array to matrix
            deltW = (1-2*sigmoid(np.dot(W, np.mat(x).T))).dot(np.mat(x))+\
            np.linalg.inv(W.T)
            # method2: use np.outer() to get array, both of them are of shape (len,)
            #deltW = np.outer(1-2*sigmoid(np.dot(W, x.T)),x)+np.linalg.inv(W.T) # np.dot((2,3),(3,))==(2,)
            W += alpha*deltW
        
    ###################################
    return W

In [5]:
def unmix(X, W):
    S = np.zeros(X.shape)

    ######### Your code here ##########
    S = X.dot(W.T) # since now audio matrix is opposed to it in lecture, we use W.T to recover S
    ##################################
    return S

In [6]:
X = normalize(load_data())

In [7]:
for i in range(X.shape[1]):
        print('Playing mixed track %d' % i)
        play(X[:, i]) # each column corresponding to one of the mixed signals xi

Playing mixed track 0
Playing mixed track 1
Playing mixed track 2
Playing mixed track 3
Playing mixed track 4


In [8]:
%time W = unmixer(X)


Separating tracks ...
we are at alpha: 0.1
we are at alpha: 0.1
we are at alpha: 0.1
we are at alpha: 0.05
we are at alpha: 0.05
we are at alpha: 0.05
we are at alpha: 0.02
we are at alpha: 0.02
we are at alpha: 0.01
we are at alpha: 0.01
we are at alpha: 0.005
we are at alpha: 0.005
we are at alpha: 0.002
we are at alpha: 0.002
we are at alpha: 0.001
we are at alpha: 0.001
Wall time: 1min 15s


In [9]:
S = normalize(unmix(X, W))

In [10]:
# % clear audio are from movies:
# % * Godfather
# % * Southpark
# % * Beethoven 5th
# % * Austin Powers
# % * Matrix (the movie, not the linear algebra construct :-) 
for i in range(S.shape[1]):
        print('Playing separated track %d' % i)
        play(S[:, i])

Playing separated track 0
Playing separated track 1
Playing separated track 2
Playing separated track 3
Playing separated track 4
